# notebook

> [`notebook`](https://github.com/jupyter/notebook) is a document-per-browser-tab interactive computing environment built on `jupyterlab` 

In [ ]:
NAME = "notebook"
GH_ORG = "jupyter"
JS_NS = "@jupyter-notebook"
URL = f"https://github.com/{GH_ORG}/{NAME}"

In [ ]:
import importnb

with importnb.Notebook():
    from jupyak.tasks import __jupyterlab, __lumino
    from jupyak.tasks import _well_known as W
    from jupyak.tasks import _yak as Y

## schema

## tasks

Exclude some packages by npm name from linking. These manifest during webpack builds as:

```bash
[webpack-cli] Error: Cannot find module '@jupyterlab/nbconvert-css/package.json'
```

In [ ]:
lab_excludes = "|".join(
    [
        "help-extension",
        "inspector-extension",
        "launcher-extension",
        "logconsole-extension",
        "nbconvert-css",
        "pluginmanager-extension",
        "rendermime-extension",
        "statusbar-extension",
        "template",
    ],
)
js_dep_exclude_patterns = [rf"^@jupyterlab/({lab_excludes})$"]

Exclude some packages from being built and linked into other repos. 

In [ ]:
js_dist_exclude_patterns = [
    "/_metapackage",
    "/app",
    "/buildutils",
]

In [ ]:
ext = "packages/lab-extension"
mp = "packages/_metapackage"
mp_tsbi = f"{mp}/{W.TSCONFIG_BUILDINFO}"
ext = "packages/lab-extension"
py_ext = "notebook/labextension"
js_bundle = "notebook/static/bundle.js"
py_ext_style = f"{py_ext}/static/style.js"
ext_tsbi = f"{ext}/{W.TSCONFIG_BUILDINFO}"

In [ ]:
JS_DIST_TASKS = {
    "./packages/_metapackage/": [
        {
            "name": "meta:build",
            "actions": [["yarn", "build"]],
            "file_dep": [
                f"../lumino/packages/*/{W.TSCONFIG_BUILDINFO}",
                f"../jupyterlab/packages/*/{W.TSCONFIG_BUILDINFO}",
                "packages/*/src/**/*.ts",
                "packages/*/src/**/*.tsx",
            ],
            "targets": [mp_tsbi, ext_tsbi],
        },
    ],
    "./app/": [
        {
            "name": "app:build:prod",
            "actions": [["yarn", "build:prod"]],
            "file_dep": [
                mp_tsbi,
                "packages/*/schema/*.json",
                "packages/*/style/**/*.css",
                "packages/*/style/**/*.svg",
                "packages/*/style/**/*.js",
            ],
            "targets": [js_bundle],
        },
    ],
}

In [ ]:
LAB_EXT = {
    f"./{ext}/": {
        "file_dep": [ext_tsbi],
        "targets": [f"{py_ext}/{W.PACKAGE_JSON}", py_ext_style],
    },
}

In [ ]:
DEFAULTS = {
    "name": NAME,
    "github": {"url": URL},
    "py": {
        "dependencies": [
            __jupyterlab.NAME,
        ],
        "lab_extensions": LAB_EXT,
        "file_dep": [
            js_bundle,
            py_ext_style,
        ],
    },
    "js": {
        "dependencies": [
            __lumino.NAME,
            __jupyterlab.NAME,
        ],
        "link_exclude_patterns": js_dep_exclude_patterns,
        "dist_exclude_patterns": js_dist_exclude_patterns,
        "tasks": JS_DIST_TASKS,
    },
}

In [ ]:
@Y.Yak.repo(
    NAME,
    tuple(DEFAULTS["py"]["dependencies"] + DEFAULTS["js"]["dependencies"]),
)
def default_factory(yak: Y.Yak):
    return DEFAULTS